In [1]:
import nltk 
from nltk.corpus import stopwords
import json
import arff # https://pypi.python.org/pypi/liac-arff
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

contractionsFile = open("english-contractions-list.txt", "r")
contractions = []
for line in contractionsFile:
    contractions = line.split(',')

stopWords = set(stopwords.words('english'))


/Users/Phil/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#object that holds a lot of our data

class NewsObject:
    id = '0'
    targetTitle = []
    targetDescription = ""
    targetKeywords = []
    targetParagraphs = []
    targetCaptions = []
    postText = []
    postMedia = []
    postTimestamp = ''
    #truthMedian = ''
    #truthMean = ''
    #truthMode = ''
    truthClass = ""
    #truthJudgments = []
    attributes = ()
    
    def __init__(self, line):
        
        self.id = line['id']
        self.targetTitle= line['targetTitle']
        self.targetKeywords = line['targetKeywords']
        self.targetParagraphs = line['targetParagraphs']
        self.targetCaptions = line['targetCaptions']
        self.postText = line['postText']
        self.postMedia = line['postMedia']
        self.postTimestamp = line['postTimestamp']
        
    def addTruth(self, line):
        #self.truthMedian = line['truthMedian']
        #self.truthMean = line['truthMean']
        #self.truthMode = line['truthMode']
        if line['truthClass'] == 'clickbait':
            self.truthClass = '1'
        else:
            self.truthClass = '0'
        #self.truthJudgments = line['truthJudgments']
        


In [3]:
#import files
instances = []

with open('dataset/instances_train.jsonl') as file:
    for line in file:
        temp = NewsObject(json.loads(line))
        instances.append(temp)
        
with open('dataset/truth_train.jsonl') as file2:
    i = 0
    for line in file2:
        instances[i].addTruth(json.loads(line))
        i += 1
#print(instances[0].targetKeywords)

In [6]:
def extractFeatures(newsObject):
    feat = {
        "wordCount": 0, "informal": False, "beginsQuestion": False, "beginsNum": False,
        "beginsThis": False, "titleStopPerc": 0, "titleProperPerc": 0, "posSent": 0,
        "neuSent": 0, "negSent": 0, "compoundSent": 0, "articleWords": 0, "titlePercVerbs": 0,
        "unigrams": 0, "bigrams": 0, "trigrams": 0, "fourgrams": 0, "fivegrams": 0, "percNouns": 0,"unigramsArticle": 0,
        "bigramsArticle": 0, "trigramsArticle": 0, "fourgramsArticle": 0, "fivegramsArticle": 0, "percAdj": 0,
        "percAdv": 0,  "percentKeywordsInTitle": 0, "sentDiffTitleBody": 0, "hasContractions": False, 
        "label": newsObject.truthClass
    }
    
    words = newsObject.postText[0].split(' ')
    keywords = newsObject.targetKeywords
    text = nltk.word_tokenize(newsObject.postText[0])
    tokenizedList = nltk.pos_tag(text)
    
    
    feat['wordCount'] = len(words)
    
    if words[0].isdigit():
        feat['beginsNum'] = True
    elif words[0].lower() == 'this':
        feat['beginsThis'] = True
    
    numSame = 0
    
    for word in words:
        if word in keywords:
            numSame += 1
        if word in contractions:
            feat['hasContractions'] = True
  
    numProper = 0
    numStop = 0
    numVerb = 0
    numNoun = 0
    numAdj = 0
    numAdv = 0
    
    if len(tokenizedList) > 0:
        firstWord = tokenizedList[0]
        if firstWord[1] == 'MD' or firstWord[1] == 'WRB':
            feat['beginsQuestion'] = True
    
    for partOfSpeech in tokenizedList:
        if partOfSpeech[1] == 'NNP':
            numProper += 1
        elif partOfSpeech[1] == 'VB' or partOfSpeech[1] == 'VBP' or partOfSpeech[1] == 'VBD' or partOfSpeech[1] == 'VBN':
            numVerb += 1
        elif partOfSpeech[1] == 'NN':
            numNoun += 1
        elif partOfSpeech[1] == 'PRP':
            f2 = True
        elif partOfSpeech[1] == 'DJ':
            numAdj += 1
        elif partOfSpeech[1] == 'RB':
            numAdv += 1
        if(partOfSpeech[0] in stopWords):
            numStop += 1

    feat['titleStopPerc'] = round(numStop/feat['wordCount'], 2)
    feat['titleProperPerc'] = round(numProper/feat['wordCount'], 2)
    feat['titlePercVerbs'] = round(numVerb/feat['wordCount'], 2)
    feat['percNouns'] = round(numNoun/feat['wordCount'], 2)
    feat['percAdj'] = round(numAdj/feat['wordCount'], 2)
    feat['percAdv'] = round(numAdv/feat['wordCount'], 2)
    feat['percentKeywordsInTitle'] = round(numSame/feat['wordCount'], 2)
    
    feat['unigrams'] = sum(Counter(ngrams(text,1)).values())
    feat['bigrams'] = sum(Counter(ngrams(text,2)).values())
    feat['trigrams'] = sum(Counter(ngrams(text,3)).values())
    feat['fourgrams'] = sum(Counter(ngrams(text,4)).values())
    feat['fivegrams'] = sum(Counter(ngrams(text,5)).values())
    
    sentence = newsObject.postText[0]
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sentence)
    feat['posSent'] = ss['pos']
    feat['neuSent'] = ss['neu']
    feat['negSent'] = ss['neg']
    feat['compoundSent'] = ss['compound']
    
    paraSent = 0
    countPara = 0
    ss2 = {}
    for item in newsObject.targetParagraphs:
        feat['articleWords'] += len(item.split())
        text = nltk.word_tokenize(item)
        feat['unigramsArticle'] += sum(Counter(ngrams(text,1)).values())
        feat['bigramsArticle'] += sum(Counter(ngrams(text,2)).values())
        feat['trigramsArticle'] += sum(Counter(ngrams(text,3)).values())
        feat['fourgramsArticle'] += sum(Counter(ngrams(text,4)).values())
        feat['fivegramsArticle'] += sum(Counter(ngrams(text,5)).values())
        ss2 = sid.polarity_scores(item)
        countPara += 1
        paraSent += ss2['compound']
        
    if countPara > 0:
        paraSent = paraSent/countPara
    
    feat['sentDiffTitleBody'] = abs(feat['compoundSent'] - paraSent)
    
    
    #do noun, adverb, adjective, proper noun analysis for text bodies
        
    featTuple = (feat['wordCount'], feat['informal'], feat['beginsQuestion'], feat['beginsNum'], 
            feat['beginsThis'], feat['titleStopPerc'], feat['titleProperPerc'], 
            feat['posSent'], feat['neuSent'], feat['negSent'], feat['compoundSent'], feat['articleWords'], 
            feat['titlePercVerbs'], feat['unigrams'], feat['bigrams'], feat['trigrams'], feat['fourgrams'], 
            feat['fivegrams'], feat['percNouns'], feat['unigramsArticle'],
            feat['bigramsArticle'], feat['trigramsArticle'], feat['fourgramsArticle'], feat['fivegramsArticle'], 
            feat['percAdj'], feat['percAdv'], feat['percentKeywordsInTitle'], 
            feat['sentDiffTitleBody'], feat['hasContractions'], feat['label'])
    newsObject.attributes = featTuple
    return newsObject
            

In [ ]:
for item in instances:
    item = extractFeatures(item)



In [ ]:
### dump to arff
features = [("word count", 'NUMERIC'),
            ("contains informal pronouns", ['True', 'False']),
            ("Begins w/ question word", ['True', 'False']),
            ("Begins w/ number", ['True', 'False']),
            ("Begins with 'this'", ['True', 'False']),
            ("percent stop words", 'NUMERIC'),
            ("Percent proper nouns", 'NUMERIC'),
            ("Pos sent", 'NUMERIC'),
            ("Neu sent", 'NUMERIC'),
            ("Neg sent", 'NUMERIC'),
            ("Compound sent", 'NUMERIC'),
            ("Article Length", 'NUMERIC'),
            ("Percent verbs", 'NUMERIC'),
            ("Unigrams", 'NUMERIC'),
            ("Bigrams", 'NUMERIC'),
            ("Trigrams", 'NUMERIC'),
            ("Fourgrams", 'NUMERIC'),
            ("Fivegrams", 'NUMERIC'),
            ("Percent nouns", 'NUMERIC'),
            ("Unigrams article body", 'NUMERIC'),
            ("Bigrams article body", 'NUMERIC'),
            ("Trigrams article body", 'NUMERIC'),
            ("Fourgrams article body", 'NUMERIC'),
            ("Fivegrams article body", 'NUMERIC'),
            ("Percent adj", 'NUMERIC'),
            ("Percent adv", 'NUMERIC'),
            ("Percent keywords in title", 'NUMERIC'),
            ("Difference in sent body v. title", 'NUMERIC'),
            ('Has contractions', ['True', 'False']),
            ("label", ['0', '1'])]
data = {}
data.setdefault('attributes', features)
data.setdefault('description', '')
data.setdefault('relation', 'clickbait_sample')
data.setdefault('data', [])
for item in instances:
    if item.attributes[28] == '1':
        print(item.postText[0])
        print(item.attributes)
    data['data'].append(item.attributes)

with open('sample_train.arff', 'w') as f:
    f.write(arff.dumps(data))

In [ ]:
text1 = nltk.word_tokenize("metal")
tokenizedList = nltk.pos_tag(text1)
print(tokenizedList)
text = nltk.word_tokenize(instances[100].postText[0])
bigrams = ngrams(text,1)
print(Counter(bigrams).values())
